In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
from yoda.alignments import load_rfam
from yoda import graphs as ygraphs

import cmcomp

data = {}
import matplotlib as mpl
mpl.rc("figure", dpi=300)

In [ ]:
a,l = load_rfam(full= False, add_cov= False)

# CMCOMPARE

In [ ]:


# THIS IS HOW WE WOULD RERUN THE CALCULATION
# names = [aa.gf["AC"][3:] for aa in a]
# [cmcomp.dumpcm(name) for name in names]
# IMPORTANTRESULT = cmcomp.run_cmcompare_pairwise()
# IMPORTANTRESULT.to_csv('cmcompare_full_run')

# LOAD CMCOMP DATA
data['CMCompare'] = cmcomp.loadcmcomp('cmcompare_full_run_2024_06_27')



# KRAID


In [ ]:
matrix = ygraphs.alignment_to_vectors(a)


'''
def mkdf(mat): 
    sim = mat @ mat.T
    # sim = -euclidean_distances(mat)
    data = [{'cm1':a, 'cm2':b , 'score1': sim[a,b]} for a in Range(l) for b in Range(l)]
    return pd.DataFrame(data)
#yoda_data = mkdf(matrix)
#yoda_dist = cmcomp.to_dist(cmcomp.pivot_numpy(yoda_data))
'''

from sklearn.metrics import euclidean_distances
def mkdist(mat):
    return  euclidean_distances(mat)
    mat = (mat@mat.T).toarray()
    mat+=abs(mat.min())
    np.fill_diagonal(mat,0)
    return mat


ourmethod = 'KRAID'
data[ourmethod] = mkdist(matrix)





In [ ]:

# one could do it like this:

# a,l = load_rfam(full= False, add_cov= False)
# graphs = lmz.Map(ali2graph.writecons, a)
#eden_sim = ygraphs.vectorize_graphs(graphs, normalization=True, min_r = 2,min_d=1, inner_normalization=True)
# data['NSPDK'] = mkdist(eden_sim)
data['NSPDK'] = mkdist(ygraphs.vectorize_graphs([alignment.graph for alignment in a]))



# INFERNAL

In [ ]:

data['Infernal']=cmcomp.infernal_tbl_to_dist(a,l,'inftools/infernal.tbl')
data['Infernal_global']=cmcomp.infernal_tbl_to_dist(a,l,'inftools/infernal_global.tbl')




# clan discovery

In [ ]:
randomarray = np.random.rand(395,395)
np.fill_diagonal(randomarray, 0)
data['random']= randomarray

In [ ]:
# k clans plot 
# we also need to collect the results for the other normed methods -> 3 x 2 lines are in this plot 

#sns.reset_defaults()
# FULL PLOT

# z = sns.color_palette("muted")
# huemap = dict(zip(np.unique(df.Method),z.as_hex()))
# print(huemap)

#sns.reset_defaults()
# REDUCED PLOT

df = cmcomp.mkHitRateData(data,l)
cmcomp.plot_hitrate_plusCSLS(df)
cmcomp.plot_hitrate_noCSLS(df)


In [ ]:
import numpy as np
from yoda.ml import nearneigh

# Example usage (assuming 'data' is defined and 'l' contains the labels)
results = cmcomp.collect_results_precrec(data, l)#%%


In [ ]:
import pandas as pd

# Constant for the plot title


# Original dataframe creation and full-data plot
df = pd.DataFrame(results)
cmcomp.plot_precision_recall_curve(df, style_column='Distances')

# Filtering the dataframe and plotting filtered data
filtered_df = df[(df['Distances'] == 'Normalized') & (df['Method'] != 'Infernal_global')]
cmcomp.plot_precision_recall_curve(filtered_df)

In [ ]:
df2 = cmcomp.make_results_table(data,l)
print(df2.to_latex(index=True,  header=True, float_format="%.2f"))

df2

In [ ]:
df2